In [5]:
import pandas as pd
import numpy as np
from pandas_datareader import data, wb
import ta as ta
import warnings
import sys, os


pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.options.mode.chained_assignment = None
warnings.filterwarnings('ignore')

class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout


## Data Wrangling

In [22]:
def get_data():
    """
    Downloads and wrangles data for BlackSwan 2.0 VIX modeling. 
  
    Combines the two index datasets and creates a 
    new DataFrame (df) that contains the target feature (Trading Days with 3 
    STDV shifts based on a rolling window)
  
    :input: vix  => CBOE VIX Historical Data
    :input: gspc => GSPC S&P 500 index with matching start date to VIX
  
    :return: df  => cominbation VIX/GSPC Dataset with Target Classifier 
    """
  
    ############### Getting Data ##################
  
    vix = (data.DataReader('^VIX', 
                           "yahoo", 
                           start='1990-1-02', 
                           end='2019-5-31')
           .drop(columns = ['Volume', 'Adj Close']))
  
    gspc = data.DataReader('^GSPC', 
                           "yahoo", 
                           start='1990-1-02',
                           end='2019-5-31')
    
    treasury = (pd.read_csv('USTREASURY-YIELD.csv')
                .sort_values(by = 'Date')
                .drop(columns=['1 MO', '2 MO', '20 YR']))
    
    fred = pd.read_csv('FRED-PERMIT.csv')
    
    ism = pd.read_csv('ismvsgdp.csv').drop(index = [0,1])
    
    lag = pd.read_csv('USLgI.csv')
    
    inv = (pd.read_csv('inversion.csv')
           [['Date','3m1s_inversion','3m2s_inversion',
             '2s10s_inversion','2s30s_inversion']])
  
    ############### Wrangling Data #################
    
    # Rename the Columns
    vix.columns      = ['vix_high', 'vix_low', 'vix_open', 'vix_close']
    gspc.columns     = ['gspc_high', 'gspc_low', 'gspc_open',
                        'gspc_close','gspc_volume','gspc_adj_close']
  
    # Join the VIX and GSPC
    df = vix.join(gspc)
  
    # Pull Date columns out of the index
    df = df.reset_index()
    
    # Merge DF with the Treasury Data on the Date Feature
    # Date needs to be converted to Datetime format to match df['Date']
    treasury['Date'] = pd.to_datetime(treasury['Date'],
                                      infer_datetime_format=True)
    
    df = pd.merge(df, treasury, how='inner', on='Date')
    
    # Merge DF with the Inversion Features Engineered by Damerei
    # Sort and create key for merging
    inv = inv[inv['Date'] < '2019-06-01'].sort_values(by='Date')
    inv['Date'] = pd.to_datetime(inv['Date'], infer_datetime_format=True)
    
    df = pd.merge(df, inv, how='inner', on='Date')
    
    # A new feature is needed to act as a key for the following features
    df['y/m'] = df['Date'].map(lambda x: x.strftime('%Y-%m'))
    
    # Line up the Date column with the new key for the FRED Data
    fred = fred[fred['Date'] > '1989-12-01'].sort_values(by='Date')
    fred['Date'] = pd.to_datetime(fred['Date'], infer_datetime_format=True)
    fred['Date'] =  fred['Date'].map(lambda x: x.strftime('%Y-%m'))
    
    # Line up the Date column iwth the new key for the ISM/GDP Data
    ism = ism[ism['ticker'] > '1989-12-01'].sort_values(by='ticker')
    ism['ticker'] = pd.to_datetime(ism['ticker'], infer_datetime_format=True)
    ism['ticker'] =  ism['ticker'].map(lambda x: x.strftime('%Y-%m'))
    # Add most recent ISM data
    ism = (ism.append(pd.DataFrame([['2019-04', 52.8, np.nan]
                                   ,['2019-05', 52.1,np.nan]]
                                   ,columns=ism.columns))
           .reset_index()
           .drop(columns='index'))
    #Add most recent GDP Value
    ism['GDP CURY Index'].iloc[351] = 5.0
    # Fill in Quarterly GDP values
    ism = ism.fillna(method='ffill').fillna(method='bfill')
    
    # Wrangle the date column for the Lagging Index Dataframe
    lag['Date'] = pd.to_datetime(lag['Date'], format='%b-%y')
    #Create a pivot point around 2019 to accomodate the 2-digit year format
    for i in np.arange(len(lag)):
        if lag['Date'][i].year > 2019:
            lag['Date'][i] = lag['Date'][i].replace(year=lag['Date'][i].year-100)
        else:
            pass
    # Map to the y/m key format
    lag['Date'] =  lag['Date'].map(lambda x: x.strftime('%Y-%m'))
    #Slice the dataframe to relevant time period
    lag = lag[lag['Date'] > '1989-12']
    

    # Create New Columns with a single iteration using list comprehension
    f  = []
    m  = []
    g  = []
    ll = []
    lg = []

    for i in np.arange(len(df)):
        # Merge the New Private Housing Units Authorized by Building Permits (FRED)
        f.append(float(fred.loc[fred['Date'] == df['y/m'][i]]['Value']
                       .values
                       .tolist()[0]))
        
        # Merge the ISM NAPMPMI Index
        m.append(float(ism.loc[ism['ticker'] == df['y/m'][i]]['NAPMPMI Index']
                       .values
                       .tolist()[0]))
        
        # Merge the GDP CURY Index
        g.append(float(ism.loc[ism['ticker'] == df['y/m'][i]]['GDP CURY Index']
                       .values
                       .tolist()[0]))
        
            # Merge the Lagging Index Level Column
        ll.append(float(lag.loc[lag['Date'] == df['y/m'][i]]['Level']
                        .values
                        .tolist()[0]))
        
        # Merge the Lagging Index Growth Column
        lg.append(float(lag.loc[lag['Date'] == df['y/m'][i]]['Growth']
                        .values
                        .tolist()[0]))
    
    df['lag_index_level'] = ll
    df['lag_index_growth'] = lg
    df['fred'] = f
    df['ism'] = m
    df['gdp_cury'] = g
    
  
    ############### Momemntum Feature Engineering ################
  
    # Awesome Oscillator
    df['mom_ao']=ta.momentum.ao(df['gspc_high'],
                                df['gspc_low'],
                                s=5,len=34,
                                fillna=True)

    # Money Flow Index
    df['mom_mf']=ta.momentum.money_flow_index(df['gspc_high'],
                                              df['gspc_low'],
                                              df['gspc_close'],
                                              df['gspc_volume'],
                                              n=14,fillna=True)
  
    # Relative Strength Index
    df['mom_rsi'] = ta.momentum.rsi(df['gspc_close'],
                                    n=14,
                                    fillna=True)
  
    # Stochasitc Oscillator
    df['mom_stoch']=ta.momentum.stoch(df['gspc_high'],
                                      df['gspc_low'],
                                      df['gspc_close'],
                                      n=14,
                                      fillna=True)
  
    # Stochasitc Signal
    df['mom_st_sig']=ta.momentum.stoch_signal(df['gspc_high'],
                                              df['gspc_low'],
                                              df['gspc_close'],
                                              n=14,
                                              d_n=3,
                                              fillna=True)
  
    # True Strength Indicator
    df['mom_tsi'] = ta.momentum.tsi(df['gspc_close'],
                                    r=25,
                                    s=13,
                                    fillna=True)
  
    # Ultimate Oscillator
    df['mom_uo'] = ta.momentum.uo(df['gspc_high'],
                                  df['gspc_low'],
                                  df['gspc_close'], 
                                  s=7, 
                                  m=14, 
                                  len=28, 
                                  ws=4.0, 
                                  wm=2.0, 
                                  wl=1.0,
                                  fillna=True)
  
    # Williams %R
    df['mom_wr']=ta.momentum.wr(df['gspc_high'],
                                df['gspc_low'],
                                df['gspc_close'],
                                lbp=14,fillna=True)
  
    ############### Volume Feature Engineering ####################
  
    # Accumulation/Distribution Index
    df['vol_adi']=ta.volume.acc_dist_index(df['gspc_high'],
                                           df['gspc_low'],
                                           df['gspc_close'],
                                           df['gspc_volume'],
                                           fillna=True)
  
    # Chaikin Money Flow
    df['vol_cmf'] = ta.volume.chaikin_money_flow(df['gspc_high'],
                                                 df['gspc_low'],
                                                 df['gspc_close'],
                                                 df['gspc_volume'],
                                                 n=20,fillna=True)
  
    # Ease of Movement
    df['vol_eom'] = ta.volume.ease_of_movement(df['gspc_high'],
                                               df['gspc_low'],
                                               df['gspc_close'],
                                               df['gspc_volume'],
                                               n=20,fillna=True)
  
    # Force Index
    df['vol_fm'] = ta.volume.force_index(df['gspc_close'],
                                         df['gspc_volume'],
                                         n=2,fillna=True)
  
    # Negative Volume Index
    df['vol_nvi'] = ta.volume.negative_volume_index(df['gspc_close'],
                                                    df['gspc_volume'],
                                                    fillna=True)
  
    # On-Balance Volume
    df['vol_obv'] = ta.volume.on_balance_volume(df['gspc_close'],
                                                df['gspc_volume'],
                                                fillna=True)
  
    # Volume-Price Trend
    df['vol_vpt'] = ta.volume.volume_price_trend(df['gspc_close'],
                                                 df['gspc_volume'],
                                                 fillna=True)
    
    ############### Volatility Feature Engineering
    
    #Average True Range
    df['atr_low'] = ta.volatility.average_true_range(df['gspc_high'],
                                                     df['gspc_low'],
                                                     df['gspc_close'],
                                                     n=23)
    
    df['atr_high'] = ta.volatility.average_true_range(df['gspc_high'],
                                                      df['gspc_low'],
                                                      df['gspc_close'],
                                                      n=37)

    ############### Target Creation #################
    
    # List Comprehension to create a Percent Moved 
    p = []

    for i in np.arange(len(df)):
                # Day 2 Close   subtracted by  Day 1 Close   div by Day 1 Close  * 100
        p.append(((df['gspc_close'].shift(1)[i] - df['gspc_close'][i]) / df['gspc_close'][i])*100)
    
    df['percent_move'] = p
    
    # Create a stdev column based on the VIX
    df['stdev'] = ((df['vix_close']/np.sqrt(256))).rolling(252).mean()
    
    df['percent_move'] = np.abs(df['percent_move']).shift(1)
    
    ########################### TARGET FEATURE #############################################
    df['3_sigma_event'] = np.where(np.abs(df['percent_move']) > 3 * df['stdev'], 1, 0)
    ########################### TARGET FEATURE #############################################
    
  
     # Determine daily market movement between Close and Close
    df['vix_move']  = (1 - df['vix_close']
                       .shift(1)/df['vix_close'])
  
    df['gspc_move'] = (1 - df['gspc_close']
                        .shift(1)/df['gspc_close'])
  
    ############## Handling Null Values ##################
    
    # Interpolating the Null Values for 30yr Treasury Bonds
    df['30 YR'] = (df['30 YR'].interpolate(method='spline',
                                           order=4))

    # Drop the rest
    df = df.dropna()
    
    # Reset the index for mistake free slicing
    df = df.reset_index().drop(columns = 'index')
  
    return df
  

In [7]:
df = get_data()

In [8]:
print(df.shape,'\n', df.isnull().sum(),'\n',
      df.info(),'\n', df.describe(),'\n\n')
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7095 entries, 0 to 7094
Data columns (total 52 columns):
Date                7095 non-null datetime64[ns]
vix_high            7095 non-null float64
vix_low             7095 non-null float64
vix_open            7095 non-null float64
vix_close           7095 non-null float64
gspc_high           7095 non-null float64
gspc_low            7095 non-null float64
gspc_open           7095 non-null float64
gspc_close          7095 non-null float64
gspc_volume         7095 non-null int64
gspc_adj_close      7095 non-null float64
3 MO                7095 non-null float64
6 MO                7095 non-null float64
1 YR                7095 non-null float64
2 YR                7095 non-null float64
3 YR                7095 non-null float64
5 YR                7095 non-null float64
7 YR                7095 non-null float64
10 YR               7095 non-null float64
30 YR               7095 non-null float64
3m1s_inversion      7095 non-null bool
3m2s_inve

,Date,vix_high,vix_low,vix_open,vix_close,gspc_high,gspc_low,gspc_open,gspc_close,gspc_volume,gspc_adj_close,3 MO,6 MO,1 YR,2 YR,3 YR,5 YR,7 YR,10 YR,30 YR,3m1s_inversion,3m2s_inversion,2s10s_inversion,2s30s_inversion,y/m,lag_index_level,lag_index_growth,fred,ism,gdp_cury,mom_ao,mom_mf,mom_rsi,mom_stoch,mom_st_sig,mom_tsi,mom_uo,mom_wr,vol_adi,vol_cmf,vol_eom,vol_fm,vol_nvi,vol_obv,vol_vpt,atr_low,atr_high,percent_move,stdev,3_sigma_event,vix_move,gspc_move
0,1991-01-03,27.930000,27.930000,27.930000,27.930000,326.529999,321.899994,326.459991,321.910004,141450000,321.910004,6.64,6.71,6.72,7.08,7.27,7.56,7.86,7.93,8.11,False,False,False,False,1991-01,104.1,2.1,786.0,39.2,4.5,3.569002,46.537378,30.616455,0.094700,32.283007,8.595531,38.908782,-99.905300,-2.671184e+08,0.129211,-6.297124e-09,-2.270291e+08,804.790977,2.401960e+09,-3.408863e+06,3.760180,4.133586,1.154844,1.440880,0,0.046903,-0.014103
1,1991-01-04,27.190001,27.190001,27.190001,27.190001,322.350006,318.869995,321.910004,321.000000,140820000,321.000000,6.73,6.82,6.83,7.17,7.37,7.65,7.94,8.02,8.20,False,False,False,False,1991-01,104.1,2.1,786.0,39.2,4.5,1.730324,47.151724,28.796672,15.661794,13.533556,7.840947,42.658298,-84.338206,-1.092754e+08,0.073116,-1.429087e-08,-7.924318e+07,802.515923,2.261140e+09,-2.365257e+06,3.747999,4.115922,1.410335,1.443348,0,-0.027216,-0.002835
2,1991-01-07,28.950001,28.950001,28.950001,28.950001,320.970001,315.440002,320.970001,315.440002,130610000,315.440002,6.71,6.84,6.84,7.20,7.43,7.75,8.04,8.13,8.32,False,False,False,False,1991-01,104.1,2.1,786.0,39.2,4.5,-0.222910,47.759075,20.293223,0.000000,5.252164,6.517319,35.161492,-100.000000,-9.904705e+07,0.098190,-2.308569e-08,7.013481e+07,788.615653,2.130530e+09,-2.660361e+06,3.826782,4.154951,0.283490,1.446017,0,0.060794,-0.017626
3,1991-01-08,30.379999,30.379999,30.379999,30.379999,316.970001,313.790009,315.440002,314.899994,143390000,314.899994,6.64,6.74,6.75,7.15,7.39,7.74,8.06,8.16,8.37,False,False,False,False,1991-01,104.1,2.1,786.0,39.2,4.5,-2.875352,47.255924,19.643185,5.942108,7.201301,5.262544,36.594105,-94.057892,-1.738986e+08,0.089016,-2.323096e-08,-1.567702e+07,788.615653,1.987140e+09,-2.507751e+06,3.798660,4.128600,1.762617,1.448785,0,0.047070,-0.001715
4,1991-01-09,33.299999,33.299999,33.299999,33.299999,320.730011,310.929993,314.899994,311.489990,191100000,311.489990,6.44,6.61,6.68,7.10,7.46,7.81,8.12,8.25,8.46,False,False,False,False,1991-01,104.1,2.1,786.0,39.2,4.5,-5.357588,37.837654,15.926122,2.599802,2.847303,3.743329,26.979367,-97.400198,-2.125488e+08,-0.017458,-2.174151e-08,-2.389362e+08,788.615653,1.796040e+09,-2.314865e+06,4.059589,4.281882,0.171486,1.452056,0,0.087688,-0.010947


## Target and Features

>**Targets** I have created a number of target columns in order to create a really rough gradient descent for rolling Standard Deviations of day-day GSPC closes. 

>**Leakage** These are features to be dropped as they are deemed to be leaky or noisy

>**features** This is the resulting dataframe to be inserted into a model. 


In [9]:
df['3_sigma_event'].value_counts()

0    7035
1      60
Name: 3_sigma_event, dtype: int64

In [10]:
df['3_sigma_event'].value_counts(normalize=True)*100

0    99.154334
1     0.845666
Name: 3_sigma_event, dtype: float64

In [11]:
target  = '3_sigma_event'

to_drop  = ['Date','y/m','percent_move','mom_wr','stdev','gspc_adj_close']

features  = (df.drop(columns = target)
            .drop(columns = to_drop)
            .columns)

## Modified Regressor

In [12]:
# Model
from xgboost import XGBRegressor
# Regression metrics
from sklearn.metrics import mean_squared_error, r2_score
# Classification Metrics
from sklearn.metrics import recall_score, precision_score, f1_score, confusion_matrix, accuracy_score

In [13]:
model   = XGBRegressor (base_score=0.72, booster='gbtree', colsample_bylevel=1,
                        colsample_bytree=1, gamma=0, learning_rate=0.13, max_delta_step=0,
                        max_depth=8, min_child_weight=.8, missing=None, n_estimators=180,
                        n_jobs=-1, objective='reg:squarederror',random_state=42, reg_alpha=0, 
                        reg_lambda=1, scale_pos_weight=1,seed=42, silent=True, subsample=1, 
                        eval_metric='rmse', normalize_type='forest')

In [14]:
def split(cut, features=features, df=df):
    """
    Temporal Train Test Split. Holdout is most recent data.  
    """
    
    # Creates X and y features. 
    # Shifts the target to a day later. 
    X = df[features]
    y = df['percent_move']
    
    # Train Test Split: 70% Train, 30% Test
    X_train = X[:int(len(X.index)*cut)]
    X_test= X[int(len(X.index)*cut):len(X.index)]
    y_train=y[:int(len(y.index)*cut)]
    y_test=y[int(len(y.index)*cut):len(y.index)]
    
    return X_train, X_test, y_train, y_test
    

In [15]:
def second_split(cut_2, X_train, X_test, y_train, y_test):
    """
    Splits Train-Test data into a smaller set for stretching
    gradient decent. 
    """
    frames = [X_train, X_test, y_train, y_test]
    
    for frame in frames:
        frame = frame.reset_index().drop(columns = 'index')
    
    # Train Test Split: 70% Train, 30% Test
    X_train_2 = X_train[:int(len(X_train.index)*cut_2)]
    X_test_2= X_train[int(len(X_train.index)*cut_2):len(X_train.index)]
    y_train_2=y_train[:int(len(y_train.index)*cut_2)]
    y_test_2=y_train[int(len(y_train.index)*cut_2):len(y_train.index)]
    
    return X_train_2, X_test_2, y_train_2, y_test_2
    

In [16]:
def bootstrap_gradient(metric, X_train, X_test, y_train, y_test,
                       X_train_2, X_test_2, y_train_2, y_test_2,
                       y_pred):
    
    # Grab the Standard Deviation and target data
    # slice it based on the training data
    stdevs = df.iloc[len(X_train_2):len(X_train)]['stdev'].values
    three_sigma = df.iloc[len(X_train_2):len(X_train)]['3_sigma_event'].values
    highest_score = 0
    raise_by = 0
    
    # Brute force a gradient descent
    
    for j in np.arange(start=1, stop=3, step=.001):   
        y_pred_mod = []
        for i in np.arange(len(y_pred)):    
            if y_pred[i] < y_test_2.values[i]:
                y_pred_mod.append(y_pred[i]**j)
            else:
                y_pred_mod.append(y_pred[i])
            
        # Build a class prediction
        modified_prediction = np.array(y_pred_mod)
        class_pred = (np.where(np.divide(modified_prediction, 
                                         stdevs) >= 3, 1, 0))
        score = metric(class_pred, three_sigma)
        
        if score > highest_score:
            highest_score = score
            associated_precision = precision_score(class_pred, three_sigma)
            associated_recall = recall_score(class_pred, three_sigma)
            associated_F1 = f1_score(class_pred, three_sigma)
            raise_by = j
        else:
            pass
    
    print("Best Training F1 Score:  ", associated_F1)
    print("Best Training Precision: ", associated_precision)
    print("Best Training Recall:    ", associated_recall)
    print('Stretchification Weight: ', raise_by)
    return raise_by
    

In [17]:
def run_model(cut, cut_2, reference, model=model, df=df):
    assert cut  > 0
    assert cut < 1
    assert cut_2 > 0
    assert cut_2 < 1
    
    
    X_train, X_test, y_train, y_test = split(cut)
    
    X_train_2, X_test_2, y_train_2, y_test_2 = second_split(cut_2,
                                                            X_train, 
                                                            X_test, 
                                                            y_train, 
                                                            y_test)
    
    print('################################################################')
    print('##################### TEST/TRAIN SPLITS ########################','\n')
    print('Shapes of Training Set:    ',X_train.shape, 
          X_test.shape, y_train.shape, y_test.shape)
    print('Shapes of Training Subset: ',X_train_2.shape, 
          X_test_2.shape, y_train_2.shape, y_test_2.shape,
          '\n\n')
    print('####################### SUBSET MODEL ###########################','\n')
    
    model.fit(X_train_2, y_train_2)
    
    y_pred = np.abs(model.predict(X_test_2))
    
    print("TrainingSubset R^2 Score:", r2_score(y_test_2, y_pred),'\n')
    
    weight = bootstrap_gradient(f1_score, X_train, X_test, y_train, y_test,
                                X_train_2, X_test_2, y_train_2, y_test_2,
                                y_pred)
    
    model.fit(X_train, y_train)
    
    final_prediction = model.predict(X_test)
    
    weighted_predict = (np.abs(final_prediction))**weight
    
    stdev = df['stdev'][int(len(df.index)*cut):len(df.index)].values
    class_pred_wei = (np.where(np.divide(weighted_predict, stdev) >= 3, 1, 0))
    three_sigma_val = df['3_sigma_event'][int(len(df.index)*cut):len(df.index)].values
    
    f1 = f1_score(three_sigma_val, class_pred_wei)
    prec = precision_score(three_sigma_val, class_pred_wei)
    rec = recall_score(three_sigma_val, class_pred_wei)
    
    if reference == 'final':
        print('\n\n##################### FINAL MODEL #############################','\n')
        print("Optimal F1:        " , f1)
        print("Optimal Precision: ", prec)
        print("Optimal Recall:    ", rec)
        return model

    else:
        return f1, prec, rec
    

In [18]:
def find_best():
    
    cut_range  = np.arange(start=.7, stop=.95, step=0.1)
    f1_scores  = []
    precision  = []
    recall     = []
    train_cut  = []
    subset_cut = []
    for i in cut_range:
        for j in cut_range: 
            with HiddenPrints():
                f1, prec, rec = run_model(i, j, reference='nope')
                
                f1_scores.append(f1)
                precision.append(prec)
                recall.append(rec)
                train_cut.append(i)
                subset_cut.append(j)
    
    results = pd.DataFrame([f1_scores, precision, recall,
                            train_cut, subset_cut])
    
    return results
                
                
                

In [19]:
results = find_best()

In [20]:
cut = results.T.sort_values(0, ascending=False).iloc[0][3]
cut_2 = results.T.sort_values(0, ascending=False).iloc[0][4]

In [21]:
model = run_model(cut, cut_2, reference='final')

################################################################
##################### TEST/TRAIN SPLITS ######################## 

Shapes of Training Set:     (6385, 45) (710, 45) (6385,) (710,)
Shapes of Training Subset:  (5108, 45) (1277, 45) (5108,) (1277,) 


####################### SUBSET MODEL ########################### 

TrainingSubset R^2 Score: 0.648141317865416 

Best Training F1 Score:   0.7272727272727273
Best Training Precision:  0.7272727272727273
Best Training Recall:     0.7272727272727273
Stretchification Weight:  1.2349999999999741


##################### FINAL MODEL ############################# 

Optimal F1:         0.631578947368421
Optimal Precision:  0.6666666666666666
Optimal Recall:     0.6
